In [0]:
%%bash
pip install -q transformers
pip install seqeval

  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=2d6af33930de3a12a7593c1588fdc7d9caca031b286b402c7af8d455241789db
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


In [0]:
#!wget http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_pt.tar.gz

In [0]:
#!tar -xvf rubert_cased_L-12_H-768_A-12_pt.tar.gz

In [0]:
#!ls -lr

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path 

import os

import torch
import torch.optim as optim
import copy
import random 

# transformers
import transformers
from transformers import BertModel, BertTokenizer, BertConfig
from transformers import CamembertModel, CamembertTokenizer, CamembertConfig
from transformers import FlaubertModel, FlaubertTokenizer, FlaubertConfig

In [0]:
train = pd.read_csv('train_v2.csv')
dev = pd.read_csv('dev_v2.csv')
test = pd.read_csv('test_v2.csv')
print(train.shape, dev.shape, test.shape)

(1753, 2) (195, 2) (485, 2)


In [0]:
train.head(2)

,tweet,class
0,Deux Xanax dans la gueule pour un voyage plus ...,0
1,J'étais pas au courant de cet effet du coup j'...,0


In [0]:
x_train = list(train['tweet'])
y_train = list(train['class'])

x_dev = list(dev['cleaned_tweet'])
y_dev = list(dev['class'])

x_test = list(test['cleaned_tweet'])
y_test = list(test['class'])

In [0]:
MAX_LEN = 200
bs = 32
model = 'camembert' #'flaubert'#'camembert'
model_name = 'camembert-base' #'flaubert-base-cased' # 'camembert-base' #'bert-base-multilingual-cased'#'biobert_large' #roberta-large

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print(device)

cuda


In [0]:
if model == 'camembert':
  tokenizer = CamembertTokenizer.from_pretrained(model_name, do_lower_case=False)
elif model == 'bert':
  tokenizer = BertTokenizer.from_pretrained(model_name)
else:
  tokenizer = FlaubertTokenizer.from_pretrained(model_name, do_lower_case=False)
print(tokenizer.cls_token, tokenizer.cls_token_id)
print(tokenizer.pad_token, tokenizer.pad_token_id)
print(tokenizer.sep_token, tokenizer.sep_token_id)
print(tokenizer.unk_token, tokenizer.unk_token_id)

<s> 5
<pad> 1
</s> 6
<unk> 3


In [0]:
def tokenize_list(lst, tokenizer):
  tmp = []
  for i in lst:
    if model == 'camembert':
      tokens = tokenizer.tokenize(i)
    elif model == 'bert':
      tokens = tokenizer.tokenize(i)
    else:
      tokens = tokenizer.tokenize(i)
    tokens = [tokenizer.cls_token] + tokens + [tokenizer.sep_token]
    tmp.append(tokens)
  return tmp

In [0]:
x_train_tokenized = tokenize_list(x_train, tokenizer)
x_dev_tokenized = tokenize_list(x_dev, tokenizer)
x_test_tokenized = tokenize_list(x_test, tokenizer)
#x_test2_tokenized = tokenize_list(x_test2, tokenizer)
print(len(x_train_tokenized), len(x_dev_tokenized), len(x_test_tokenized))#, len(x_test2_tokenized))
print(x_train[0])
print(" ".join(x_train_tokenized[0]))

1753 195 485
Deux Xanax dans la gueule pour un voyage plus sympa. 10 minutes de bus j’me sens déjà partir
<s> ▁Deux ▁X ana x ▁dans ▁la ▁gueule ▁pour ▁un ▁voyage ▁plus ▁sympa . ▁10 ▁minutes ▁de ▁bus ▁j ’ me ▁sens ▁déjà ▁partir </s>


In [0]:
from keras.preprocessing.sequence import pad_sequences

pad_value = tokenizer.pad_token_id
train_input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in x_train_tokenized],
                          maxlen=MAX_LEN, dtype="long", truncating="post", 
                          padding="post", value = pad_value)
dev_input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in x_dev_tokenized],
                          maxlen=MAX_LEN, dtype="long", truncating="post", 
                          padding="post", value = pad_value)
test_input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in x_test_tokenized],
                          maxlen=MAX_LEN, dtype="long", truncating="post", 
                          padding="post", value = pad_value)

print(train_input_ids.shape, dev_input_ids.shape, test_input_ids.shape)

(1753, 200) (195, 200) (485, 200)


In [0]:
train_input_ids[0]

array([   5, 1922, 1001, 1889,  290,   29,   13, 5099,   24,   23,  805,
         40, 2248,    9,  239,  609,    8, 2485,   76,   12,  573,  437,
        235,  350,    6,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,   

In [0]:
pad_value = tokenizer.pad_token_id
train_attention_masks = np.array([[int(i != pad_value)for i in ii] for ii in train_input_ids])
dev_attention_masks = np.array([[int(i != pad_value) for i in ii] for ii in dev_input_ids])
test_attention_masks = np.array([[int(i != pad_value)for i in ii] for ii in test_input_ids])
print(train_attention_masks.shape, dev_attention_masks.shape, test_attention_masks.shape)

(1753, 200) (195, 200) (485, 200)


In [0]:
train_attention_masks[0]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])

In [0]:
train_segment_ids = np.array([[0]*MAX_LEN for ii in train_input_ids])
dev_segment_ids = np.array([[0]*MAX_LEN for ii in dev_input_ids])
test_segment_ids = np.array([[0]*MAX_LEN for ii in test_input_ids])

print(train_segment_ids.shape, dev_segment_ids.shape, test_segment_ids.shape)

(1753, 200) (195, 200) (485, 200)


In [0]:
tr_inputs = torch.tensor(train_input_ids)
val_inputs = torch.tensor(dev_input_ids)
te_inputs = torch.tensor(test_input_ids)

tr_masks = torch.tensor(train_attention_masks)
val_masks = torch.tensor(dev_attention_masks)
te_masks = torch.tensor(test_attention_masks)

tr_segment = torch.tensor(train_segment_ids)
val_segment = torch.tensor(dev_segment_ids)
te_segment = torch.tensor(test_segment_ids)

tr_class = torch.tensor(y_train)
val_class = torch.tensor(y_dev)
te_class = torch.tensor(y_test)

In [0]:
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)#, tr_class)
train_data = TensorDataset(tr_inputs, tr_masks, tr_segment, tr_class)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

#valid_data = TensorDataset(val_inputs, val_masks, val_tags)#, val_class)
valid_data = TensorDataset(val_inputs, val_masks, val_segment, val_class)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

#test_data = TensorDataset(te_inputs, te_masks, te_tags)#, te_class)
test_data = TensorDataset(te_inputs, te_masks, te_segment, te_class)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=bs)


In [0]:
from torch.nn.utils.rnn import pad_sequence

class SequenceClassifier(torch.nn.Module):
  def __init__(self, transformer_model, config):
    super().__init__()
    self.transformer = transformer_model
    if config.model_type == 'flaubert':
      self.model_type = config.model_type
      self.out = torch.nn.Linear(config.emb_dim, 2)
      self.dropout = torch.nn.Dropout(config.dropout)
    else:
      self.model_type = 'bert'
      self.out = torch.nn.Linear(config.hidden_size*2, 2)
      self.dropout = torch.nn.Dropout(config.hidden_dropout_prob)
  
  def forward(self, input_ids, attention_mask, segment_id, classification_labels=None):
    
    #Batch max length
    max_length = (attention_mask != 0).max(0)[0].nonzero()[-1].item()+1

    if max_length < input_ids.shape[1]:
      input_ids = input_ids[:, :max_length]
      attention_mask = attention_mask[:, :max_length]

    segment_id = torch.zeros_like(attention_mask)
    hidden = self.transformer(input_ids = input_ids,attention_mask = attention_mask, 
                              token_type_ids = segment_id)
    #Aggregating all the hidden states
    if self.model_type == 'bert':
      token_hidden = hidden[2]
      token_hidden = torch.mean(torch.mean(torch.stack(token_hidden), dim=0), 
                              dim=1)

      classifier_hidden = hidden[1]
      hidden_cat = token_hidden
      hidden_cat = torch.cat([token_hidden, classifier_hidden], dim=1)
    else:
      token_hidden = hidden[1]
      token_hidden = torch.mean(torch.mean(torch.stack(token_hidden), dim=0), 
                                dim=1)
      hidden_cat = token_hidden

    classification_logits = self.out(self.dropout(hidden_cat))
    outputs = [classification_logits]
    if classification_labels is not None:
      loss_fct_classification = torch.nn.CrossEntropyLoss()

      loss_classification = loss_fct_classification(classification_logits.view(-1, 2), 
                                                      classification_labels.view(-1))

      outputs += [loss_classification]
    return outputs

In [0]:
if model == 'camembert':
  config = CamembertConfig.from_pretrained(model_name)
elif model == 'bert':
  config = BertConfig.from_pretrained(model_name)
else:
  config = FlaubertConfig.from_pretrained(model_name)
config.output_hidden_states = True

In [0]:
if model == 'camembert':
  transformer_model = CamembertModel.from_pretrained(model_name, config=config)
elif model == 'bert':
  transformer_model = BertModel.from_pretrained(model_name, config=config)
else:
  transformer_model = FlaubertModel.from_pretrained(model_name, config=config)

In [0]:
for name,param in transformer_model.named_parameters():
  if 'pooler' in name:
    param.requires_grad=True
  else:
    param.requires_grad=False
  #print(name,param.requires_grad)

In [0]:
model = SequenceClassifier(transformer_model, config)

In [0]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
model.cuda();

In [0]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=2e-4)

In [0]:
from seqeval.metrics import f1_score

def flat_accuracy_classification(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,confusion_matrix

epochs = 5
max_grad_norm = 2.0
model.zero_grad()
best_f1 = 0
for _ in tqdm(range(epochs)):
  # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for batch in tqdm(train_dataloader):

      batch = tuple(t.to(device) for t in batch)
      b_input_ids, b_input_mask, b_segment, b_classes = batch
      
      outputs = model(b_input_ids, b_input_mask, b_segment, b_classes)
      
      loss = outputs[1]
      
      loss.backward()

      tr_loss += loss.item()
      nb_tr_examples += b_input_ids.size(0)
      nb_tr_steps += 1

      torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), 
                                     max_norm=max_grad_norm)

      optimizer.step()
      model.zero_grad()

    print("Train loss: {}".format(tr_loss/nb_tr_steps))

    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy_class = 0, 0
    
    nb_eval_steps, nb_eval_examples = 0, 0
    class_preds ,class_true_labels = [], []
    

    for batch in valid_dataloader:
      batch = tuple(t.to(device) for t in batch)
      b_input_ids, b_input_mask, b_segment, b_classes = batch

      with torch.no_grad():
        outputs = model(b_input_ids, b_input_mask, b_segment, b_classes)
        eval_loss = classification_loss = outputs[1]

      classification_logits = outputs[0].detach().cpu().numpy()
      class_preds.extend(list(np.argmax(classification_logits,axis=1)))
      
      class_label_ids = b_classes.to('cpu').numpy()
      
      class_true_labels.append(class_label_ids)
      
      tmp_eval_accuracy_class = flat_accuracy_classification(classification_logits, 
                                              class_label_ids)
      eval_loss += eval_loss.mean().item()

      eval_accuracy_class += tmp_eval_accuracy_class
      nb_eval_examples += b_input_ids.size(0)
      nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy Classifier: {}".format(eval_accuracy_class/nb_eval_steps))
    valid_tags = [l_i for l in class_true_labels for l_i in l ]
    pred_tags = [p for p in class_preds]
    print("Val F1 ",f1_score(valid_tags, pred_tags))
    print("Val Precision ",precision_score(valid_tags, pred_tags))
    print("Val Recall ",recall_score(valid_tags, pred_tags))
    print("Val Accuracy ",accuracy_score(valid_tags, pred_tags))
    print("Confusion matrix \n", confusion_matrix(valid_tags, pred_tags))
    if f1_score(valid_tags, pred_tags) >= best_f1:
      best_f1 = f1_score(valid_tags, pred_tags)
      print("Saving this model")
      torch.save(model.state_dict(), './best_model_fr_state_dict')
      torch.save(model, './best_model_fr')
      # print("Copying to G drive")
      # !cp ./best_model_fr_state_dict drive/My\ Drive/
      # !cp ./best_model_fr drive/My\ Drive/
      # print("Copied to G drive")
      





  0%|          | 0/5 [00:00<?, ?it/s]



  0%|          | 0/55 [00:00<?, ?it/s]



  2%|▏         | 1/55 [00:00<00:11,  4.64it/s]



  4%|▎         | 2/55 [00:00<00:11,  4.77it/s]



  5%|▌         | 3/55 [00:00<00:10,  5.05it/s]



  7%|▋         | 4/55 [00:00<00:09,  5.11it/s]



  9%|▉         | 5/55 [00:00<00:09,  5.29it/s]



 11%|█         | 6/55 [00:01<00:09,  5.43it/s]



 13%|█▎        | 7/55 [00:01<00:08,  5.54it/s]



 15%|█▍        | 8/55 [00:01<00:08,  5.72it/s]



 16%|█▋        | 9/55 [00:01<00:07,  5.77it/s]



 18%|█▊        | 10/55 [00:01<00:07,  5.86it/s]



 20%|██        | 11/55 [00:01<00:07,  5.86it/s]



 22%|██▏       | 12/55 [00:02<00:07,  5.94it/s]



 24%|██▎       | 13/55 [00:02<00:07,  5.92it/s]



 25%|██▌       | 14/55 [00:02<00:06,  6.02it/s]



 27%|██▋       | 15/55 [00:02<00:06,  5.99it/s]



 29%|██▉       | 16/55 [00:02<00:06,  5.60it/s]



 31%|███       | 17/55 [00:02<00:06,  5.79it/s]



 33%|███▎      | 18/55 [00:03<00:06,  5.77it/s]



 35%|

Train loss: 0.1814084935256026
Validation loss: 0.0032159716356545687
Validation Accuracy Classifier: 0.9866071428571429
Val F1  0.0
Val Precision  0.0
Val Recall  0.0
Val Accuracy  0.9846153846153847
Confusion matrix 
 [[192   0]
 [  3   0]]
Saving this model


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type SequenceClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "



 20%|██        | 1/5 [00:13<00:54, 13.56s/it]



  0%|          | 0/55 [00:00<?, ?it/s]



  2%|▏         | 1/55 [00:00<00:12,  4.48it/s]



  4%|▎         | 2/55 [00:00<00:11,  4.78it/s]



  5%|▌         | 3/55 [00:00<00:10,  5.03it/s]



  7%|▋         | 4/55 [00:00<00:09,  5.21it/s]



  9%|▉         | 5/55 [00:00<00:09,  5.43it/s]



 11%|█         | 6/55 [00:01<00:08,  5.62it/s]



 13%|█▎        | 7/55 [00:01<00:08,  5.75it/s]



 15%|█▍     

Train loss: 0.09341102361001752
Validation loss: 0.004388661589473486
Validation Accuracy Classifier: 0.9866071428571429
Val F1  0.0
Val Precision  0.0
Val Recall  0.0
Val Accuracy  0.9846153846153847
Confusion matrix 
 [[192   0]
 [  3   0]]
Saving this model





 40%|████      | 2/5 [00:27<00:40, 13.57s/it]



  0%|          | 0/55 [00:00<?, ?it/s]



  2%|▏         | 1/55 [00:00<00:09,  5.43it/s]



  4%|▎         | 2/55 [00:00<00:09,  5.58it/s]



  5%|▌         | 3/55 [00:00<00:09,  5.66it/s]



  7%|▋         | 4/55 [00:00<00:09,  5.66it/s]



  9%|▉         | 5/55 [00:00<00:09,  5.50it/s]



 11%|█         | 6/55 [00:01<00:08,  5.66it/s]



 13%|█▎        | 7/55 [00:01<00:08,  5.73it/s]



 15%|█▍        | 8/55 [00:01<00:08,  5.73it/s]



 16%|█▋        | 9/55 [00:01<00:07,  5.84it/s]



 18%|█▊        | 10/55 [00:01<00:07,  5.94it/s]



 20%|██        | 11/55 [00:01<00:07,  5.87it/s]



 22%|██▏       | 12/55 [00:02<00:07,  5.65it/s]



 24%|██▎       | 13/55 [00:02<00:07,  5.70it/s]



 25%|██▌       | 14/55 [00:02<00:07,  5.85it/s]



 27%|██▋       | 15/55 [00:02<00:06,  5.82it/s]



 29%|██▉       | 16/55 [00:02<00:06,  5.73it/s]



 31%|███       | 17/55 [00:02<00:06,  5.81it/s]



 33%|███▎      | 18/55 [00:03<00:06,  5.44it/s]


...




 60%|██████    | 3/5 [00:47<00:31, 15.52s/it]



  0%|          | 0/55 [00:00<?, ?it/s]



  2%|▏         | 1/55 [00:00<00:08,  6.00it/s]



  4%|▎         | 2/55 [00:00<00:09,  5.69it/s]



  5%|▌         | 3/55 [00:00<00:09,  5.75it/s]



  7%|▋         | 4/55 [00:00<00:08,  5.74it/s]



  9%|▉         | 5/55 [00:00<00:08,  5.78it/s]



 11%|█         | 6/55 [00:01<00:08,  5.79it/s]



 13%|█▎        | 7/55 [00:01<00:08,  5.87it/s]



 15%|█▍        | 8/55 [00:01<00:08,  5.82it/s]



 16%|█▋        | 9/55 [00:01<00:07,  5.85it/s]



 18%|█▊        | 10/55 [00:01<00:07,  5.83it/s]



 20%|██        | 11/55 [00:01<00:07,  5.67it/s]



 22%|██▏       | 12/55 [00:02<00:07,  5.76it/s]



 24%|██▎       | 13/55 [00:02<00:07,  5.83it/s]



 25%|██▌       | 14/55 [00:02<00:07,  5.75it/s]



 27%|██▋       | 15/55 [00:02<00:07,  5.71it/s]



 29%|██▉       | 16/55 [00:02<00:06,  5.76it/s]



 31%|███       | 17/55 [00:02<00:06,  5.77it/s]



 33%|███▎      | 18/55 [00:03<00:06,  5.68it/s]


Train loss: 0.08296241760253906
Validation loss: 0.004582609515637159
Validation Accuracy Classifier: 0.9866071428571429
Val F1  0.0
Val Precision  0.0
Val Recall  0.0
Val Accuracy  0.9846153846153847
Confusion matrix 
 [[192   0]
 [  3   0]]
Saving this model





 80%|████████  | 4/5 [01:07<00:16, 16.97s/it]



  0%|          | 0/55 [00:00<?, ?it/s]



  2%|▏         | 1/55 [00:00<00:10,  5.35it/s]



  4%|▎         | 2/55 [00:00<00:09,  5.42it/s]



  5%|▌         | 3/55 [00:00<00:09,  5.56it/s]



  7%|▋         | 4/55 [00:00<00:09,  5.60it/s]



  9%|▉         | 5/55 [00:00<00:08,  5.74it/s]



 11%|█         | 6/55 [00:01<00:08,  5.69it/s]



 13%|█▎        | 7/55 [00:01<00:08,  5.68it/s]



 15%|█▍        | 8/55 [00:01<00:08,  5.70it/s]



 16%|█▋        | 9/55 [00:01<00:08,  5.71it/s]



 18%|█▊        | 10/55 [00:01<00:07,  5.68it/s]



 20%|██        | 11/55 [00:01<00:07,  5.80it/s]



 22%|██▏       | 12/55 [00:02<00:07,  5.88it/s]



 24%|██▎       | 13/55 [00:02<00:07,  5.82it/s]



 25%|██▌       | 14/55 [00:02<00:07,  5.84it/s]



 27%|██▋       | 15/55 [00:02<00:06,  5.80it/s]



 29%|██▉       | 16/55 [00:02<00:06,  5.62it/s]



 31%|███       | 17/55 [00:02<00:06,  5.72it/s]



 33%|███▎      | 18/55 [00:03<00:06,  5.73it/s]


Train loss: 0.07685615847056562
Validation loss: 0.0053917341865599155
Validation Accuracy Classifier: 0.9866071428571429
Val F1  0.0
Val Precision  0.0
Val Recall  0.0
Val Accuracy  0.9846153846153847
Confusion matrix 
 [[192   0]
 [  3   0]]
Saving this model





100%|██████████| 5/5 [01:29<00:00, 18.00s/it]


In [0]:
model = torch.load('./best_model_fr')
for name,param in model.named_parameters():
  param.requires_grad = False
model.cuda();

In [0]:
model.eval()
eval_loss, eval_accuracy_class = 0, 0

nb_eval_steps, nb_eval_examples = 0, 0
class_preds ,class_true_labels = [], []

for batch in test_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_segment, b_classes = batch

    with torch.no_grad():
      outputs = model(b_input_ids, b_input_mask, b_segment, b_classes)
      eval_loss = classification_loss = outputs[1]

    classification_logits = outputs[0].detach().cpu().numpy()
    class_preds.extend(list(np.argmax(classification_logits,axis=1)))
    
    class_label_ids = b_classes.to('cpu').numpy()
    
    class_true_labels.append(class_label_ids)
    
    tmp_eval_accuracy_class = flat_accuracy_classification(classification_logits, 
                                            class_label_ids)
    eval_loss += eval_loss.mean().item()

    eval_accuracy_class += tmp_eval_accuracy_class
    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1
eval_loss = eval_loss/nb_eval_steps
print("Validation loss: {}".format(eval_loss))
print("Validation Accuracy Classifier: {}".format(eval_accuracy_class/nb_eval_steps))
valid_tags = [l_i for l in class_true_labels for l_i in l ]
pred_tags = [p for p in class_preds]
print("Val F1 ",f1_score(valid_tags, pred_tags))
print("Val Precision ",precision_score(valid_tags, pred_tags))
print("Val Recall ",recall_score(valid_tags, pred_tags))
print("Val Accuracy ",accuracy_score(valid_tags, pred_tags))
print("Confusion matrix \n", confusion_matrix(valid_tags, pred_tags))


Validation loss: 0.0009604394435882568
Validation Accuracy Classifier: 0.984375
Val F1  0.0
Val Precision  0.0
Val Recall  0.0
Val Accuracy  0.9835051546391752
Confusion matrix 
 [[477   0]
 [  8   0]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
